<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Требуется защитить данные клиентов страховой компании «Хоть потоп». Разрабатывается такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновыется корректность предложенного работы.

Важным условием при преобразовании явяется не ухучшение качества моделей машинного обучения. Исспользуется линейная модель. 

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score

In [2]:
data=pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [25]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [29]:
data.duplicated().sum()

153

**Вывод** по предварительному анализу данных. 
Пропусков и аномалий в данных не обнаружено. Типы данных соответсвующие. Есть дубликаты но это может быть вполне случайностью, а не ошибкой. Поэтому дубликаты оставляем.


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**
Нет, неизменится. Но нужно убедится, что численные погрешности операции не приводят к значимой погрешности.
При малых ненулевых значениях признаков $\ll 1$ либо сушественно большиз $\gg 1$ неоходимо провести масштабирование перед кодированием. В рассматриваевом примере это делать не нужно.

**Обоснование:**  
Обозначим за матрицу кодирующию признаки как $k$. Размер матрицы на единицу больше количества признаков.
В качестве такой матрицы подойдет любая случайная матрица $k$, но необходимо убедится в ее обратимости. 
Тогда закодированая матрица признаков равна
$$\tilde X=X\cdot k\,, \label{eq:X} \tag{1}$$  
исходя из того что бы хотим что бы предсказания были теми же новые параметры линейной регрессии
$$\tilde w =k^{-1}\cdot w\,.  \label{eq:tw} \tag{2}$$   
Таким образом предсказание модели будет алгебраически индентично исходному:
$$\tilde a=\tilde X\cdot\tilde w=X\cdot k\cdot k^{-1}\cdot w=X\cdot w=a$$

С другой стороны использую закодированные данные \eqref{eq:X} и тот же целевой признак мы получим туже  связь \eqref{eq:tw} на новые и старые параметры:
$$
\tilde w = \arg\min_\tilde w MSE(\tilde X \tilde w, y) = \arg\min_{k^{-1}w} MSE(X w, y)= k^{-1}\arg\min_{w} MSE(X w, y)=k^{-1}w
$$


In [4]:
np.random.seed(0)
key_size=data.shape[1]
key=np.random.rand(key_size,key_size)
print(key)

[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773   0.96366276 0.38344152]
 [0.79172504 0.52889492 0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985 0.77815675 0.87001215]
 [0.97861834 0.79915856 0.46147936 0.78052918 0.11827443]]


проверка на обратимость

In [5]:
unit=key@np.linalg.inv(key)
print(unit)
print(unit.astype('int'))

[[ 1.00000000e+00 -4.51121570e-17  1.19367574e-16  1.68874368e-16
  -6.16633644e-16]
 [-3.80775728e-16  1.00000000e+00  7.80229454e-16 -3.14226001e-16
   8.85933895e-17]
 [-6.03713657e-16 -4.94948315e-18  1.00000000e+00 -1.61628958e-16
  -2.06518729e-17]
 [-4.96469687e-17  3.03691958e-16 -6.43896092e-16  1.00000000e+00
   6.20775396e-17]
 [-7.99806765e-16 -1.37004457e-16  2.28906265e-16  3.52666888e-17
   1.00000000e+00]]
[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]]


In [6]:
encoded=data.head(5).values@key
print((encoded@np.linalg.inv(key)-data.head(5).values)[0])
print((encoded@np.linalg.inv(key))[0])
print('Отличие от исходных данных c с точностью до 10**(-10)')
((encoded@np.linalg.inv(key)-data.head(5).values)>10**(-10)).astype('int')

[-3.65224517e-11 -2.36610731e-12  0.00000000e+00  2.63877808e-12
  1.38462701e-11]
[1.00000000e+00 4.10000000e+01 4.96000000e+04 1.00000000e+00
 1.38462701e-11]
Отличие от исходных данных c с точностью до 10**(-10)


array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

## Алгоритм преобразования

**Алгоритм**

1. Кодируем только признаки  $\tilde F = F\kappa$ без единичного элемента. Учитывая, что полная матрица признаков содержит единичный столбц:$$X_{ij}=\delta_{j0}+(1-\delta_{j0})F_{i,j+1}$$
для матрицы $X$ будет другой ключ  $\tilde X = Xk$,
где ключ может быть представлен в блочном виде и связан с ключем $\kappa$:
$$
k=\begin{pmatrix} 1 & 0 \\ 0 & \kappa \end{pmatrix} \,.
$$
При генерации ключа $\kappa$ используем нормальное распределения $\sim {\cal N}(0,1)$, но можно использовать и унимодальное распределение с интервалом $[-1,1]$.

2. Проверям, что детерминант матрицы не нулевой $\det \kappa=\det k \neq 0$, что гарантирует обратимость ключа и как следствие работу алгоритма. Если матрица не обратима, геренируем новый ключ.
3. Обучаем линейную модель $\tilde w = (\tilde X^T \tilde X)^{-1} \tilde X^T y$
4. Использыуем модель для получения предсказаний  $a=\tilde X \tilde w$ на основе закодированных данных $\tilde X$

**Обоснование**

Качество регрессии не изменится, т.к. Предсказание модели алгебраически индентично исходному если обе модели обучены на одних и тех же данных:
$$\tilde a=\tilde X\cdot\tilde w=X\cdot k\cdot k^{-1}\cdot w=X\cdot w=a$$


## Проверка алгоритма

In [7]:
np.random.seed(777)
key_size=data.shape[1]-1
#key=np.random.rand(key_size,key_size)
key=np.random.normal(0, 1, (key_size,key_size))
print('Ключ кодирования')
print(key)

Ключ кодирования
[[-0.46820879 -0.82282485 -0.0653801  -0.71336192]
 [ 0.90635089  0.76623673  0.82605407 -1.32368279]
 [-1.75244452  1.00244907  0.54480945  1.8951609 ]
 [-0.76935745 -1.40309592 -0.63246751 -0.55887367]]


In [8]:
if np.linalg.det(key)!=0:
    print('Ключ можно использовать')
else:
    print('Выбирите другой ключ')

Ключ можно использовать


Для ключа исспользуем диагонально (строго) доминнантную матрицу для генирации обратимой матрицицы.

In [13]:
np.random.seed(777)
key_size=data.shape[1]-1
m = np.random.rand(key_size, key_size)
mx = np.sum(np.abs(m), axis=1)
np.fill_diagonal(m, mx)
assert np.linalg.det(m)!=0, "Ошибка ключа. Ключ не обратим"

In [14]:
features=data.drop(['Страховые выплаты'], axis=1).values
target=data[['Страховые выплаты']].values

In [15]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)

In [16]:
features_encoded=features@key
model_encoded = LinearRegression()
model_encoded.fit(features_encoded, target)
predictions_encoded = model_encoded.predict(features_encoded)

In [17]:
r2=r2_score(target,predictions)

In [18]:
r2_encoded=r2_score(target,predictions_encoded)

In [19]:
r2

0.4249455028666801

In [20]:
(1-r2_encoded/r2)*100

1.6764367671839864e-12

в общем случае желательно использовать масштобирование но в данном случае в этом нет необходимости

In [21]:
scaler=StandardScaler()
scaler.fit(features)
features_scaled=scaler.transform(features)
model.fit(features_scaled, target)
predictions = model.predict(features_scaled)
r2=r2_score(target,predictions)
print(r2)

0.42494550286668


In [22]:
scaler_encoded=StandardScaler()
scaler_encoded.fit(features_scaled@key)
features_encoded=scaler_encoded.transform(features@key)
model_encoded.fit(features_encoded, target)
predictions_encoded = model_encoded.predict(features_encoded)
r2_encoded=r2_score(target,predictions_encoded)
print(r2_encoded)

0.4249455028666679


In [23]:
(1-r2_encoded/r2)*100

2.8532731732866523e-12

**Вывод** 

В проекте рассмотрен варинт кодирования персональных данных посредством умножения вектора признаков каждой записи на квадратную матрицую. Квадратная матрица выбирается случайным образом и проверяется ее обратимость.  Такой варинт кодирования позволяет обучить линейную модель на закодированных данных. На рассмотренных данных такое кодирование и модель позволяют получить описание даннх с тойже точностью что и модель построенная на основе исходных данных. Малые отклонения в точности возможны в обе стороны в связи с численными погрешностями вычислений. Алгебраически оба подхода имеют индентичное предсказания и как следствие точность. 

Использование маштабирования данных перед обучением модели важно в общем случаее, но для представленных данных значения не имеет. 